<a href="https://colab.research.google.com/github/onism/MyLearning/blob/master/NLPBasic_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
!unzip trainingandtestdata.zip

--2020-07-05 06:32:07--  https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81363704 (78M) [application/zip]
Saving to: ‘trainingandtestdata.zip’

trainingandtestdata 100%[===================>]  77.59M  29.6MB/s    in 2.6s    

2020-07-05 06:32:10 (29.6 MB/s) - ‘trainingandtestdata.zip’ saved [81363704/81363704]

Archive:  trainingandtestdata.zip
  inflating: testdata.manual.2009.06.14.csv  
  inflating: training.1600000.processed.noemoticon.csv  


In [3]:
import pandas as pd

tweetsdf = pd.read_csv('training.1600000.processed.noemoticon.csv', header=None)

UnicodeDecodeError: ignored

Error: UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 80-81: invalid continuation byte

Solution: It appears that the default C-based CSV parser that pandas uses does not like some ot the unicode in the filter, so we need to switch to the python-based parser


In [4]:
tweetsdf = pd.read_csv('training.1600000.processed.noemoticon.csv', header=None, engine='python')

In [5]:
tweetsdf.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [7]:
tweetsdf[0].value_counts()

4    800000
0    800000
Name: 0, dtype: int64

In [8]:
tweetsdf['sentiment_cat'] = tweetsdf[0].astype('category')

In [9]:
tweetsdf.head()

,0,1,2,3,4,5,sentiment_cat
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,0
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,0
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,0
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",0


In [10]:
tweetsdf['sentiment'] = tweetsdf['sentiment_cat'].cat.codes
tweetsdf.head()

,0,1,2,3,4,5,sentiment_cat,sentiment
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,0
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,0,0
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,0,0
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,0,0
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",0,0


In [11]:
tweetsdf['sentiment'].value_counts()

1    800000
0    800000
Name: sentiment, dtype: int64

In [12]:
tweetsdf.to_csv('train-processed.csv', header=None, index=None)
tweetsdf.sample(10000).to_csv('train-processed-sample.csv', header=None, index=None)

In [13]:
from torchtext import data       

In [15]:
LABEL = data.LabelField()
TWEET = data.Field(tokenize='spacy', lower=True)

In [24]:
fields = [ ('score', None), ('id', None), ('Time', None), ('query', None), ('name', None), ('tweet', TWEET), ('category', None), ('label', LABEL)  ]

tweetsDataset = data.TabularDataset(path='train-processed.csv', format='csv', fields=fields, skip_header=False)

In [25]:
train, test, valid = tweetsDataset.split(split_ratio=[0.8,0.1,0.1])

In [26]:
vars(train.examples[10])

{'label': '0',
 'tweet': ['@ashdonaldson',
  'wow',
  ',',
  'serious',
  'exodus',
  'from',
  'london',
  'now',
  '.',
  'got',
  'ta',
  'b',
  'really',
  'lucky',
  'to',
  'keep',
  'ur',
  'job',
  'there',
  '.',
  'dunno',
  'if',
  'bris',
  'is',
  'going',
  '2',
  'be',
  'better']}

In [27]:
vocab_size = 20000
TWEET.build_vocab(train, max_size=vocab_size)
len(TWEET.vocab)

20002

In [28]:
TWEET.vocab.freqs.most_common(10)

[('i', 799005),
 ('!', 724301),
 ('.', 648003),
 (' ', 470024),
 ('to', 452676),
 ('the', 417665),
 (',', 386611),
 ('a', 304898),
 ('my', 253021),
 ('it', 242441)]

In [31]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
(train, valid, test), 
batch_size = 32,
sort_key = lambda x: len(x.tweet),
sort_within_batch = False)

In [32]:
import torch.nn as nn
class FirstLSTM(nn.Module):
    def __init__(self, hidden_size, embedding_dim, vocab_size):
        super(FirstLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.encoder = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=1)
        self.predict = nn.Linear(hidden_size,2)
    
    def forward(self, seq):
        output, (hidden,_) = self.encoder(self.embedding(seq))
        preds = self.predict(hidden.squeeze(0))
        return preds

model = FirstLSTM(100, 300, 20002)

In [33]:
from torch import optim      

In [34]:
optimizer = optim.Adam(model.parameters(), lr=2e-2)
criterion = nn.CrossEntropyLoss()

def train(epochs, model, optimizer, criterion, train_iterator, valid_iterator):
    for epoch in range(1, epochs+1):
        training_loss = 0
        valid_loss = 0
        for batch_idx, batch in enumerate(train_iterator):
            optimizer.zero_grad()
            predict = model(batch.tweet)
            loss = criterion(predict, batch.label)
            loss.backward()
            optimizer.step()
            training_loss += loss.item()